In [1]:
from phonecodes import phonecodes
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
from typing import Iterable, List
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader, Dataset
from timeit import default_timer as timer
from torch.nn import Transformer
from torch import Tensor
from sklearn.model_selection import train_test_split
import copy
import tqdm
import librosa
import seaborn as sns
import torch.nn as nn
import torch
import torch.nn.functional as F
import numpy as np
import math
import os
import pandas as pd
import matplotlib.pyplot as plt
import textgrid
from scipy.spatial.distance import euclidean
import plotly.graph_objects as go
import pandas as pd
import jiwer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import nltk
nltk.download('cmudict')
from nltk.corpus import cmudict
from transformers import AutoProcessor, AutoModelForCTC
from phonemizer.backend.espeak.wrapper import EspeakWrapper
import soundfile as sf

_ESPEAK_LIBRARY = r"C:\Program Files\eSpeak NG\libespeak-ng.dll"
EspeakWrapper.set_library(_ESPEAK_LIBRARY)
'''processor_P = AutoProcessor.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")
model_P = AutoModelForCTC.from_pretrained("facebook/wav2vec2-lv-60-espeak-cv-ft")'''


from transformers import WhisperProcessor, WhisperForConditionalGeneration

# 加载 Whisper 处理器
processor_W = WhisperProcessor.from_pretrained("openai/whisper-small")

# 加载 Whisper 模型
model_W = WhisperForConditionalGeneration.from_pretrained("openai/whisper-small")

[nltk_data] Downloading package cmudict to
[nltk_data]     C:\Users\Alex\AppData\Roaming\nltk_data...
[nltk_data]   Package cmudict is already up-to-date!


c:\Users\Alex\anaconda3\envs\BayesPCN\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/1.97k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/967M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/3.87k [00:00<?, ?B/s]

In [13]:
def get_pathset(paths):
    return [os.path.join(dir, each_file) for dir, mid, files in os.walk(paths) for each_file in files if each_file.endswith(".wav")]
audio_dir =r"..\Nov10\speech_files"
set1_list=[0,1,2,3,4,5,6,7,8,9,10,12,13,14,15,16]
set2_list=[17,18,19,20,21,22,24,25,26,27,28,29,30,31,37,40]
    
audio_path=get_pathset(audio_dir)
audio, sr = librosa.load(audio_path[0])
wave_res = librosa.resample(audio, orig_sr=sr, target_sr=16000)
tg = textgrid.TextGrid.fromFile(audio_path[0][:-3]+"TextGrid")
tg_sentence = tg[0]
for _,i in enumerate(tg[0]):
    if i.mark!="":
        tg_sentence[_-1].maxTime=tg_sentence[_].minTime
tg_sentence = [i for i in tg_sentence if i.mark!=""]
tg_sentence=[tg_sentence[i] for i in set1_list+set2_list]
tg_word = [i for i in tg[1] if i.mark!="" and i.mark!="sp"]
model_W.to(device)
for _,each_sentence in enumerate(tg_sentence):
    each_sentence_list=[]
    start_sentence = int(each_sentence.minTime*16000)
    end_sentence = int(each_sentence.maxTime*16000)
    input=processor_W(wave_res[start_sentence:end_sentence], sampling_rate=16000, return_tensors="pt").input_features.to(device)
    with torch.no_grad():
        predicted_ids = model_W.generate(input)
    
        #out_encoder=model_W(input).logits
        #out_FE=model.wav2vec2.feature_extractor(input)[0].transpose(1,0).cpu().numpy()
    #outind=torch.argmax(out_encoder,dim=-1).cpu().numpy()
    #out_encoder=copy.deepcopy(out_encoder).cpu().numpy()
    break

In [16]:
decoder_input_ids = torch.tensor([[model_W.config.decoder_start_token_id]]).to(device)  # 起始 token
outputs = model_W(input, decoder_input_ids=decoder_input_ids, output_hidden_states=True)
print(outputs.logits)

tensor([[[0.6508, 2.9931, 5.6374,  ..., 3.1381, 3.2454, 3.7221]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)


In [8]:
transcription = processor_W.batch_decode(predicted_ids, skip_special_tokens=True)
print(transcription)

[' A boy fell from the window.']


In [20]:
outputs.logits

tensor([[[0.6508, 2.9931, 5.6374,  ..., 3.1381, 3.2454, 3.7221]]],
       device='cuda:0', grad_fn=<UnsafeViewBackward0>)